# scrapy_crawlspider读书网

CrawlSpider 案例

需求：读书网数据入库

1. 创建项目：scrapy startproject 项目的名字  
2. 跳转到spiders路径：cd ./项目的名字/项目的名字/spiders
3. 创建爬虫类：scrapy genspider -t crawl 爬虫文件的名字 爬取的网站的域名
4. items
5. spiders
6. settings
7. pipelines

    数据保存到本地  
    数据保存到mysql数据库

In [ ]:
scrapy startproject scrapy_readbook_051

cd scrapy_readbook_051/scrapy_readbook_051/spiders

# 创建爬虫文件
scrapy genspider -t crawl read https://www.dushu.com/book/1188.html


In [ ]:
# scrapy_readbook_051/scrapy_readbook_051/items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class ScrapyReadbook051Item(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    name = scrapy.Field()
    src = scrapy.Field()
    pass


In [ ]:
# scrapy_readbook_051/scrapy_readbook_051/spiders/read.py


import scrapy
from scrapy.linkextractors import LinkExtractor
from scrapy.spiders import CrawlSpider, Rule
from scrapy_readbook_051.items import ScrapyReadbook051Item

class ReadSpider(CrawlSpider):
    name = "read"
    allowed_domains = ["www.dushu.com"]

    # 有坑，第一页经常不符合我们后面设置的匹配规则，导致最后爬取的少了第一页的数据
    # start_urls = ["https://www.dushu.com/book/1188.html"]
    start_urls = ["https://www.dushu.com/book/1188_1.html"]

    # 在本例中follow=False只爬到了前13页的数据
    # follow=True可以爬取所有的数据
    rules = (Rule(LinkExtractor(allow=r"/book/1188_\d+\.html"), callback="parse_item", follow=False),)

    def parse_item(self, response):
        img_list = response.xpath('//div[@class="bookslist"]//img')

        for img in img_list:
            name = img.xpath('./@alt').extract_first()
            src = img.xpath('./@data-original').extract_first()
            
            book = ScrapyReadbook051Item(name=name, src=src)
            yield book


到settings.py中打开管道

In [ ]:
# scrapy_readbook_051/scrapy_readbook_051/pipelines.py

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter


class ScrapyReadbook051Pipeline:
    def open_spider(self, spider):
        self.fp = open('book.json', 'w', encoding='utf-8')


    def process_item(self, item, spider):
        self.fp.write(str(item))
        
        return item
    

    def close_spider(self, spider):
        self.fp.close()
